In [ ]:
!pip install transformers accelerate einops langchain langchain-community langchain-chroma unstructured xformers bitsandbytes sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
model_id = "IlyaGusev/saiga_llama3_8b"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

llm = HuggingFacePipeline(pipeline=query_pipeline)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
loader = DirectoryLoader('./documents/', glob="*.md", show_progress=True)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, chunk_overlap=100, add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

print(docs[0])
print(all_splits[0])
print(all_splits[1])

  0%|          | 0/7 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 7/7 [00:05<00:00,  1.21it/s]

page_content='Структура: Аграрно-технологический институт, Институт биохимической технологии и нанотехнологии, Медицинский институт, Инженерная академия, Факультет физико-математических и естественных наук, Факультет гуманитарных и социальных наук, Филологический факультет, Факультет русского языка и общеобразовательных дисциплин, Экономический факультет, Юридический институт, Общеуниверситетские кафедры, Институт гостиничного бизнеса и туризма, Институт иностранных языков, Учебно-научный институт гравитации и космологии, Институт мировой экономики и бизнеса (Международная школа бизнеса), Институт медико-биологических проблем, Институт русского языка, Институт инновационно-образовательных программ в здравоохранении, Институт восточной медицины, Институт экологии, Научно-образовательный Институт современных языков, межкультурной коммуникации и миграций (ИСЯМКиМ, Телеграм-канал Института), Научно-исследовательский институт развития мозга и высших достижений, Филиал РУДН в городе Сочи Кра

In [ ]:
# Define the path to the pre-trained model you want to use
modelPath = "cointegrated/rubert-tiny2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("Год основания?")

print(len(retrieved_docs))
print(retrieved_docs)
print(retrieved_docs[0].page_content)

3
[Document(page_content='Год основания Российского университета дружбы народов: 1960\nПрезидент РУДН: Филиппов Владимир Михайлович\nРектор РУДН: Ястребов Олег Александрович\nКоличество студентов в РУДН: 23 388 (2017): 19 350 очников, 1957 вечерников, 2081 заочников\nКоличество иностранных студентов в РУДН: 5556 (2017 год)\nРасположение РУДН: Москва\nБлижайшая станция метро к РУДН: Университет Дружбы Народов (строится)\nАдрес кампуса РУДН: Москва, улица Миклухо-Маклая, 3—21\nЮридический адрес РУДН: Москва, улица Миклухо-Маклая, 6\nСайт РУДН: rudn.ru\nНаграды РУДН: Орден Дружбы народов, Медаль ЮНЕСКО имени Альберта Эйнштейна', metadata={'source': 'documents/1.md', 'start_index': 512}), Document(page_content='Общежития университета признаны лучшими в 2011 году по итогам конкурса «Наш студенческий дом», проводимого Департаментом семейной и молодёжной политики города Москвы.', metadata={'source': 'documents/6.md', 'start_index': 955}), Document(page_content='В 2010 году университет был наг

In [ ]:
rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> Ты ассистент, специализирующийся на вопросах, касающихся Российского университета дружбы народов (РУДН).
    Если вопрос не относится к РУДН, то ты должен вежливо отказать в помощи. Любые вопросы не про Российский университет дружбы народов должны остаться без ответа.
    Используй предложенные фрагменты контекста для формирования ответов. Если ответа на вопрос в контексте нет, скажи, что затрудняешься ответить. Если не уверен в ответе, признай это.
    Ответы должны быть развёрнутыми и лаконичными, но не более трех предложений. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "context"],
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

questions = [
    "Какого цвета бывают кошки?",
    "Кто является ректором Российского университета дружбы народов?",
    "В честь кого назван РУДН?",
    "В честь кого назван российский университет дружбы народов?",
    "Чем собака отличается от кошки?",
    "Что ты можешь рассказать про РУДН?",
    "Что ты можешь рассказать про российский университет дружбы народов?",
    "РУДН это университет или детский сад?",
    "Что ты можешь рассказать про Москву?",
    "Сколько иностранных студентов в РУДН?",
    "Кто ты?"
]

qa_chain.invoke(questions[0])

# for chunk in qa_chain.stream(questions[0]):
#     print(chunk, end="", flush=True)

NameError: name 'PromptTemplate' is not defined